# Intent 와 NER 모델을 만들기 위한 Data의 구성 방법

In [99]:
train_data_order = ['판교에 오늘 피자 주문해줘']
train_data_reserve = ['오늘 날짜에 호텔 예약 해줄레']
train_data_info = ['오늘 날짜에 판교 여행 정보 알려줘']

dict_entity = {
    'date' : ['오늘','내일','모래'],
    'loc' : ['판교','야탑','서현'],
    'menu' : ['피자','햄버거','치킨'],
    'hotel' : ['호텔','여관','민박'],
    'travel' : ['여행','놀러갈','구경']
}

length = 1
for key in list(dict_entity.keys()):
    length = length * len(dict_entity[key])
print(length)

243


# Vector 구성 (a~z / 한글)
 - 일반적으로 처리단위가 작아질수록 미등록어에서 자유롭고 작은 vector 차원을 유지할 수 있지만
 - 문장의 길이가 길어지고, 학습이 어려워지는 문제가 있기에 적절한 embedding을 찾아야하는데 
 - 이부분은 Biz Domain 별 차이가 있음 복잡도나 표현 가능성등을 적절한 균형에서 찾아야함 

In [100]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
vocab = (" $%'()+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ"
           "\\^_abcdefghijklmnopqrstuvwxyz{|}")

# 형태소 분석

In [101]:
from konlpy.tag import Mecab
mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic')
morpphed_text = mecab.pos(train_data_order[0])
print(morpphed_text)

[('판교', 'NNG'), ('에', 'JKB'), ('오늘', 'MAG'), ('피자', 'NNG'), ('주문', 'NNG'), ('해', 'XSV+EC'), ('줘', 'VX+EC')]


# Feature Engineering (명사만 도출)

Feature Engineering으로 Intent와 NER의 정확도를 높임)
 - 일반명사(NNG) [메뉴]
 - 고유명사(NNP) [지역]
 - 영어(SL) [Pizza]
 - 시간부사(MAG) [오늘, 내일, 모래]
 - 한국어 품사 태그 비교표 https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0

In [102]:
tagged_text = ''
for pos_tags in morpphed_text:
    if (pos_tags[1] in ['NNG','MAG', 'NNP','SL'] and len(pos_tags[0]) > 1): #Check only Noun
        feature_value = pos_tags[0]
        tagged_text = tagged_text + pos_tags[0] + ' '
print(tagged_text)

판교 오늘 피자 주문 


# Intent 학습 Data의 구성 (Text Representation)

- Intent 성능 향상을 위해 parse한 Text Data를 represent화 함

In [103]:
pattern = ''
for word in tagged_text.split(' '):
    entity = list(filter(lambda key:word in dict_entity[key],list(dict_entity.keys())))
    if(len(entity) > 0): 
        pattern = pattern + 'tag' + entity[0] + ' '
    else:
        pattern = pattern + word + ' '

print(pattern)

tagloc tagdate tagmenu 주문  


# Data augmentation (Entity and Pattern)
 - 각 의도별 Pattern text를 entity의 N 배수로 Augmenatation 작업

In [104]:
def augmentation_pattern(pattern, dict_entity):
    #입력된 패턴을 List로 바꿈
    aug_pattern = pattern.split(' ')
    #Augment된 Text List
    augmented_text_list = []
    #copy를 위한 임시 List
    temp_aug = []
    for i in range(0,len(aug_pattern)):
        #Entity에 해당하는 값일 경우 Entity List를 가져옴
        if(aug_pattern[i].find("tag") > -1):
            dict_list = dict_entity[aug_pattern[i].replace("tag","")]
            #각 Entity별로 값을 append하면서 Pattern구성
            for j in range(0,len(dict_list)):
                #최초 Entity값은 그냥 추가만함
                if(i == 0):
                    augmented_text_list.append(dict_list[j] + " ")
                elif(j == 1):
                    augmented_text_list = list(filter(lambda word:len(word.split(' ')) == i+1 ,augmented_text_list))
                    copy_data_order = augmented_text_list * (len(dict_list)-2)
                    augmented_text_list = list(map(lambda x:x + dict_list[j] + " ",augmented_text_list))
                    augmented_text_list = augmented_text_list + temp_aug + copy_data_order
                else:
                    #List의 수를 체크하여 값을 추가
                    temp_aug = list(filter(lambda word:len(word.split(' ')) == i+1 ,augmented_text_list))
                    temp_aug = list(map(lambda x:x + dict_list[j] + " " ,temp_aug))
                    #추가된 List를 위해 기존 값 삭제
                    if(j != 0):
                        augmented_text_list = augmented_text_list[0:len(augmented_text_list) - len(temp_aug)]
                    augmented_text_list = augmented_text_list + temp_aug
        #Entity추가 대상이 아닐 경우 Pattern만 추가
        else:
            augmented_text_list = list(map(lambda x:x + aug_pattern[i] + " ",augmented_text_list))
        #N*N으로 증가시키기 위한 List
        temp_aug = augmented_text_list
    return augmented_text_list



In [105]:
augmented_text_list = augmentation_pattern(pattern, dict_entity)
augmented_text_list

['판교 내일 햄버거 주문   ',
 '야탑 내일 햄버거 주문   ',
 '서현 내일 햄버거 주문   ',
 '판교 오늘 햄버거 주문   ',
 '야탑 오늘 햄버거 주문   ',
 '서현 오늘 햄버거 주문   ',
 '판교 모래 햄버거 주문   ',
 '야탑 모래 햄버거 주문   ',
 '서현 모래 햄버거 주문   ',
 '판교 내일 피자 주문   ',
 '야탑 내일 피자 주문   ',
 '서현 내일 피자 주문   ',
 '판교 오늘 피자 주문   ',
 '야탑 오늘 피자 주문   ',
 '서현 오늘 피자 주문   ',
 '판교 모래 피자 주문   ',
 '야탑 모래 피자 주문   ',
 '서현 모래 피자 주문   ',
 '판교 내일 치킨 주문   ',
 '야탑 내일 치킨 주문   ',
 '서현 내일 치킨 주문   ',
 '판교 오늘 치킨 주문   ',
 '야탑 오늘 치킨 주문   ',
 '서현 오늘 치킨 주문   ',
 '판교 모래 치킨 주문   ',
 '야탑 모래 치킨 주문   ',
 '서현 모래 치킨 주문   ']

# BIO Tagging

# NER을 위한 Full Train Text 확보